In [1]:
%pip install yfinance

     ---------------------------------------- 0.0/59.2 kB ? eta -:--:--
     ---------------------------------------- 59.2/59.2 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/112.2 kB ? eta -:--:--
     -------------------------------------- 112.2/112.2 kB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     ------------------------------------ - 122.9/129.4 kB 3.6 MB/s eta 0:00:01
     -------------------------------------- 129.4/129.4 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
     - -------------------------------------- 0.2/3.9 MB 5.3 MB/s eta 0:00:01
     -- ------------------------------------- 0.3/3.9 MB 3.5 MB/s eta 0:00:02
     ---- ----------------------------------- 0.4/3.9 MB 3.4 MB/s eta 0:00:02
     ------ --------------------------------- 0.6/3.9 MB 3.4 MB/s eta 0:00:01
     ------- -------------------------------- 0.7/3.9 MB 3.3 MB/s eta 0:00:01
 

In [8]:
import yfinance as yf
import pandas as pd

In [15]:
hist = yf.download(tickers = "btc-usd", period="max", interval="1d")
df = pd.DataFrame(hist)
df = df.reset_index()
df

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...,...
3088,2023-03-02,23647.019531,23739.138672,23245.021484,23475.466797,23475.466797,20386398516
3089,2023-03-03,23476.632812,23479.347656,22213.238281,22362.679688,22362.679688,26062404610
3090,2023-03-04,22362.923828,22405.177734,22198.980469,22353.349609,22353.349609,11166012913
3091,2023-03-05,22354.144531,22613.685547,22307.142578,22435.513672,22435.513672,13317001733
